In [3]:
using JuMP
using HiGHS
using EzXML
using Plots
using Ipopt
using Cbc

In [4]:
function read_input(file_path::String)
    lines = readlines(file_path)

    lines = filter(line -> !isempty(line) && !startswith(strip(line), "#"), lines)
    lines = map(line -> split(line, "#")[1] |> strip, lines)

    idx = 1

    M = parse(Int, lines[idx]); idx += 1
    N = parse(Int, lines[idx]); idx += 1
    K = parse(Int, lines[idx]); idx += 1
    L = parse(Int, lines[idx]); idx += 1
    T = parse(Int, lines[idx]); idx += 1
    Q = parse(Int, lines[idx]); idx += 1

    θ = parse.(Int, split(lines[idx])); idx += 1
    d = parse.(Int, split(lines[idx])); idx += 1

    α = [parse.(Int, split(lines[idx + i - 1])) for i in 1:N]; idx += N
    τ = parse.(Int, split(lines[idx])); idx += 1

    β = [parse.(Int, split(lines[idx + i - 1])) for i in 1:M]; idx += M
    ρ_k = [parse.(Int, split(lines[idx + i - 1])) for i in 1:M]; idx += M
    ρ = parse.(Int, split(lines[idx])); idx += 1

    γ = Array{Int, 3}(undef, M, K, Q)
    for q in 1:Q
        for m in 1:M
            γ[m, :, q] = parse.(Int, split(lines[idx])); idx += 1
        end
    end

    ρ_ = [parse.(Int, split(lines[idx + i - 1])) for i in 1:M]; idx += M

    return M, N, K, L, T, Q, θ, d, α, τ, β, ρ_k, ρ, γ, ρ_
end

read_input (generic function with 1 method)

In [5]:
file_path = "input/input.txt"
M, N, K, L, T, Q, θ, d, α, τ, β, ρ_k, ρ, γ, ρ_ = read_input(file_path)

λ = Array{Float64, 2}(undef, M, 2)
for m in 1:M
	λ[m, 1] = 1
	λ[m, 2] = 1
end
μ = Array{Float64, 2}(undef, M, 2)
for m in 1:M
	μ[m, 1] = 1
	μ[m, 2] = 1
end

In [6]:
function w(t::Int)
	return sum(x[t, n]*d[n]*τ[n] for n in 1:N)
end

w (generic function with 1 method)

In [7]:
function P(m::Int, t::Int)
	if (t == 1)
		return ρ[m]/100
	end
    sum_α = λ[m, 1] * sum(w(l)*(MathConstants.e^((-t-l)/μ[m, 1])) for l in 1:t-1)
    sum_β = λ[m, 2] * sum(w(l)*(MathConstants.e^((-t-l)/μ[m, 2])) for l in 1:t-1)
    return (ρ[m] + sum_α - sum_β)/100
end

P (generic function with 1 method)

In [8]:
function F(m::Int, k::Int, t::Int)
	return sum(β[m][t]*x[t, n]*α[n][k] for n in 1:N)
end

F (generic function with 1 method)

In [9]:
function R(m::Int, k::Int, t::Int)
	return F(m, k, t) * P(m, t)
end

R (generic function with 1 method)

In [10]:
function ℜ(m::Int, k::Int, q::Int)
	return sum(R(m, k, t) for t in 1:θ[q])
end

ℜ (generic function with 1 method)

In [11]:
model = Model(Cbc.Optimizer)

# Define variables
@variable(model, x[1:T, 1:N] >= 0, Int)  # x[t, n] = a на тренировочный день t упражнение n ставится a раз.
@variable(model, Ε >= 0)  # Minimum satisfaction level
@variable(model, ϵ[1:M, 1:K, 1:Q] >= 0)
@variable(model, ϵ_[1:M, 1:Q] >= 0)

# длительность упражнений в тренировочный день не превышает длительность тренировки
@constraint(model, [t=1:T], sum(x[t, n] for n in 1:N) <= L)

# эффект + штраф превышает требуемый прирост
@constraint(model, [m=1:M, k=1:K, q=1:Q], ℜ(m, k, q) + ϵ[m, k, q] >= γ[m, k, q])

# то же самое для тренированности
@constraint(model, [m=1:M, q=1:Q], P(m, θ[q]) + ϵ_[m, q] >= ρ_[m][q])

# все штрафы не выше порога
@constraint(model, [m=1:M, k=1:K, q=1:Q], ϵ[m, k, q] <= Ε)

# то же самое для штрафов тренированности
@constraint(model, [m=1:M, q=1:Q], ϵ_[m, q] <= Ε)

# Objective: Maximize the minimum satisfaction level
@objective(model, Min, Ε)

# Solve the model
optimize!(model)

# Retrieve results
println("Optimal satisfaction level: ", value(Ε))

ErrorException: Constraints of type MathOptInterface.ScalarQuadraticFunction{Float64}-in-MathOptInterface.GreaterThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

In [ ]:
function ℜ__(m::Int, k::Int, q::Int)
	return sum(R(m, k, t) for t in 1:q)
end

In [ ]:
# Visualization of P(m, t)
using Plots

p1 = plot(title="P(m, t) for all m and t", xlabel="t", ylabel="P(m, t)")
for m in 1:M
    P_values = [P(m, t) for t in 1:T]
    plot!(p1, 1:T, P_values, label="m=$m")
end
display(p1)

# Visualization of ℜ__(m, k, q)
p2 = plot(title="ℜ__(m, k, q) for all m, k, and q", xlabel="q", ylabel="ℜ__(m, k, q)")
for m in 1:M, k in 1:K
    ℜ_values = [ℜ__(m, k, q) for q in 1:T]
    plot!(p2, 1:T, ℜ_values, label="m=$m, k=$k")
end
display(p2)